# Reto Idealista
**Autor:** *David Camuñas*



Previamente instalar las dependencias del proyecto, puede hacerse ejecutando la siguiente línea o de forma manual mediante el comando:
```
$ sh setup.sh
```

## 1. (5%) Obtener el fichero usando la librería `requests`

In [10]:
from requests import get as rget

def get_web_file(url):
    """
    Función para obtener el fichero web.
    """
    file = rget(url)
    print("[X] Status code: " + str(file.status_code))
    return file

if __name__ == "__main__":
    file = get_web_file("https://gist.github.com/juanriaza/9b6f7ab3fc1cbda81100fa5d32512fd8/raw/a61db2d8d81789d5c7040eacee7b225bc2b2aa7f/data.csv.bz2")
    
    # Guardar en el directorio files el fichero web descargado
    open("./data.csv.bz2", 'wb').write(file.content)

[X] Status code: 200


---------------

## 3. (15%) Usando línea de comandos importar los datos de dicho fichero en una base de datos que resida en un fichero SQLite. [TIP](https://www.sqlite.org/cli.html#csv)  

### 1. Descomprimir archivo .bz2 y extraerlo en el directorio actual.    
```
    $ bzip2 -d data.csv.bz2
```
    
### 2. Crear y seleccionar base de datos SQLite (id_data.db).
```
    $ sqlite3 id_data.db
```

### 3. Una vez dentro de la linea de comandos de SQLITE importar el csv a la nueva base de datos
```
    $ .import ./data.csv tab_principal
```

Finalmente borrar el fichero csv, ya que no se utilizará en los siguientes apartados. Las consultas se realizarán a la base de datos creada

In [ ]:
cmd('rm data.csv')

-------------------------

## 4. (30%) Usando la librería `sqlite3` deberás responder a las siguientes preguntas mediante queries SQL:

### Función para establecer conexión con base de datos (id_data.db)

In [11]:
import sqlite3 as sqlite

sql_db = 'id_data.db'
connection = sqlite.connect(sql_db)
cursor = connection.cursor()

### Función utilizada para la realización de consultas

In [12]:
def my_sql_query(query):
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

* ### Número de inmuebles en alquiler que tiene cada cliente profesional por código postal

In [ ]:
my_sql_query('select customer_id, zipcode, count(*) from tab_principal where is_professional=1 and transaction_type="rent" group by customer_id, zipcode')

* ### Obtén los siguientes códigos postales de la ciudad de Barcelona:

* #### Código postal del precio por metro cuadrado más barato
En este caso el número de inmuebles más barato son varios debido a que si se analiza el precio por área de las pisos, varíos su precio tiene el mismo valor que su área. Además, en este caso como en el siguiente no se diferencia si el inmueble se vende o se alquila, ya que no se específica en el enunciado.


In [17]:
my_sql_query('create view price_area(customer_id, zipcode, pr_m2) as select customer_id, zipcode, price/area from tab_principal where city like("%Barcelona%") and price!=0 and area!=0 group by customer_id, zipcode')
my_sql_query('select distinct zipcode from price_area where pr_m2 in(select min(pr_m2) from price_area)')


('08004',)
('08028',)
('08024',)
('08005',)
('08012',)
('08010',)
('08015',)
('08018',)
('08013',)
('08014',)
('08003',)
('08731',)


* #### Código postal del precio por metro cuadrado más caro


In [37]:
my_sql_query('select distinct customer_id, zipcode from price_area where pr_m2 in(select max(pr_m2) from price_area)')


('39323032373531343335333136', '08027')


* ### Cliente profesional que tiene la cartera de inmuebles más envejecida
Se entiende como: realizar la suma de todas las fechas en las que cada cliente han añadido el anuncio del inmueble, y el cliente que en su totalidad de anuncios añadidos disponga de la fecha más antigua (min), será el resultado.

In [48]:
#my_sql_query('create view customer_dates(customer_id, total_dates) as select customer_id, sum(date_added) from tab_principal where is_professional=1 group by customer_id')
my_sql_query('select customer_id from customer_dates where total_dates in(select min(total_dates) from customer_dates)')

('39323032373532363535363035',)
('39323032373533343431303938',)


* ### En qué ciudad se encuentra el terreno urbanizable más caro?
En este caso, el resultado son varias ciudades, debido a que el precio del terreno más caro es el mismo en todas ellas.

In [33]:
my_sql_query('select city from tab_principal where price in (select max(price) from tab_principal) and subtypology like("%Terreno Urbanizable%")')

('Cubelles',)
('Esparreguera',)
('Vic',)


---------------------

## 5. (35%) Realiza los mismos ejercicios del apartado 4 utilizando la librería `pandas`

Importar libreria pandas

In [2]:
import pandas as pd
import sqlite3 as sqlite

sql_db = 'id_data.db'
connetion = sqlite.connect(sql_db)

### Mostrar dataframe de la tabla principal de la base de datos

In [3]:
data_df = pd.read_sql('SELECT * FROM tab_principal', connetion)
data_df

,customer_id,city,typology,subtypology,transaction_type,price,zipcode,bedrooms,bathrooms,area,area_land,is_professional,date_added
0,39323032373531333936323630,Barcelona Capital,Vivienda,Dúplex,sale,320000,08038,2,2,91,0,1,2012-05-18T12:26:00
1,39323032373532323235313938,Barcelona Capital,Vivienda,Piso,rent,5000,08034,5,3,0,0,1,2014-04-01T11:23:00
2,39323032373534303438333537,Barcelona Capital,Oficina,,rent,75,08017,0,0,50,0,1,2014-06-02T09:38:00
3,39323032373531373435393038,Dosrius,Terreno,Terreno Urbanizable,sale,75000,08319,0,0,740,0,0,2014-12-11T00:01:00
4,39323032373534353337323338,Barcelona Capital,Oficina,,rent,3520,08029,0,0,160,0,1,2015-04-08T10:26:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122369,393433303033303138363338,Pineda de Mar,Vivienda,Piso,sale,145000,08397,3,1,85,0,1,2019-12-05T17:47:00
122370,39323032373531333735313830,Olesa de Montserrat,Vivienda,Piso,sale,195000,08640,3,2,103,0,1,2019-12-05T20:47:00
122371,39323032373632343634333535,Montesquiu,Vivienda,Piso,sale,83400,08585,2,1,100,0,1,2019-12-06T06:16:00
122372,39323032373631383234353237,L'Hospitalet de Llobregat,Vivienda,Ático,sale,385000,08902,3,2,132,0,1,2019-12-06T13:18:00


* ### Número de inmuebles en alquiler que tiene cada cliente profesional por código postal

In [49]:
query = 'select customer_id, zipcode, count(*) as properties from tab_principal where is_professional=1 and transaction_type="rent" group by customer_id, zipcode'
clients_zipcode_df = pd.read_sql(query, connetion)
clients_zipcode_df

,customer_id,zipcode,properties
0,393030303830303030333431,08950,2
1,393030303830303030333431,08960,8
2,393030303830303030333431,08970,1
3,393030313430303030323137,08001,1
4,393030313430303030323137,08003,1
...,...,...,...
6775,393433303033313033373236,08339,1
6776,393433303033313033373236,08349,1
6777,393433303033313033373236,08358,2
6778,393433303033313033373236,08360,1


* ### Obtén los siguientes códigos postales de la ciudad de Barcelona:
* #### Código postal del precio por metro cuadrado más barato

In [20]:
query = 'select distinct zipcode from price_area where pr_m2 in(select min(pr_m2) from price_area)'
ziprice_min_df = pd.read_sql(query, connection)
ziprice_min_df

,zipcode
0,08004
1,08028
2,08024
3,08005
4,08012
5,08010
6,08015
7,08018
8,08013
9,08014


* #### Código postal del precio por metro cuadrado más barato

In [21]:
query = 'select distinct zipcode from price_area where pr_m2 in(select max(pr_m2) from price_area)'
ziprice_max_df = pd.read_sql(query, connection)
ziprice_max_df

,zipcode
0,08020


* ### Cliente profesional que tiene la cartera de inmuebles más envejecida


In [45]:
query ='select customer_id from customer_dates where total_dates in(select min(total_dates) from customer_dates)'
custom_old_df = pd.read_sql(query, connection)
custom_old_df


,customer_id
0,39323032373532363535363035
1,39323032373533343431303938


* ### En qué ciudad se encuentra el terreno urbanizable más caro?


In [47]:
query = 'select city from tab_principal where price in (select max(price) from tab_principal) and subtypology like("%Terreno Urbanizable%")'
expensive_land_df = pd.read_sql(query, connection)
expensive_land_df

,city
0,Cubelles
1,Esparreguera
2,Vic


----------------

## 6. (10%) Un inversor quiere saber por código postal qué propiedades son interesantes. Añade al dataframe de `pandas` las siguientes columnas:
* over_valued (0/1)
* normal(0/1)
* under_valued (0/1)

In [4]:
import random

new_data_df = data_df.assign(over_valued=pd.Series(random.choices([0,1], k=len(data_df))).values, normal=pd.Series(random.choices([0,1], k=len(data_df))).values, under_valued=pd.Series(random.choices([0,1], k=len(data_df))).values)
new_data_df

,customer_id,city,typology,subtypology,transaction_type,price,zipcode,bedrooms,bathrooms,area,area_land,is_professional,date_added,over_valued,normal,under_valued
0,39323032373531333936323630,Barcelona Capital,Vivienda,Dúplex,sale,320000,08038,2,2,91,0,1,2012-05-18T12:26:00,1,0,1
1,39323032373532323235313938,Barcelona Capital,Vivienda,Piso,rent,5000,08034,5,3,0,0,1,2014-04-01T11:23:00,0,1,1
2,39323032373534303438333537,Barcelona Capital,Oficina,,rent,75,08017,0,0,50,0,1,2014-06-02T09:38:00,1,0,0
3,39323032373531373435393038,Dosrius,Terreno,Terreno Urbanizable,sale,75000,08319,0,0,740,0,0,2014-12-11T00:01:00,1,0,0
4,39323032373534353337323338,Barcelona Capital,Oficina,,rent,3520,08029,0,0,160,0,1,2015-04-08T10:26:00,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122369,393433303033303138363338,Pineda de Mar,Vivienda,Piso,sale,145000,08397,3,1,85,0,1,2019-12-05T17:47:00,1,1,1
122370,39323032373531333735313830,Olesa de Montserrat,Vivienda,Piso,sale,195000,08640,3,2,103,0,1,2019-12-05T20:47:00,1,0,0
122371,39323032373632343634333535,Montesquiu,Vivienda,Piso,sale,83400,08585,2,1,100,0,1,2019-12-06T06:16:00,0,0,1
122372,39323032373631383234353237,L'Hospitalet de Llobregat,Vivienda,Ático,sale,385000,08902,3,2,132,0,1,2019-12-06T13:18:00,1,1,1


### Qué código postal tiene el mayor número de "chollos"?

In [45]:
import numpy as np
by_zipcode_sum = new_data_df.groupby(['zipcode']).agg({'under_valued':np.sum})
by_zipcode_sum[by_zipcode_sum.under_valued==int(by_zipcode_sum.agg({'under_valued':np.max}))]


,under_valued
zipcode,
08870,1086


## 7. (5%) Guarda el nuevo dataframe del anterior apartado en un fichero json comprimido con gzip


### Primera forma

In [ ]:
import gzip
from os import system as cmd

new_data_df.to_json('./data.json', orient='table')
file_in = open('./data.json', 'rb')
file_out = open('./data.json.gz', 'wb')
file_out.write(gzip.compress(file_in.read()))
file_out.close()
file_in.close()

cmd('rm data.json')